### Iterators

In [1]:
ints = lambda n: range(1, n+1)

In [2]:
nums = ints(5)
it = map(lambda x: x**2, nums)
print(it)

In [3]:
lst = [1, 2, 3]
it = iter(lst)
it

In [4]:
iter(it)

In [5]:
print(next(it))

1


Exhausting iterator:

In [6]:
for _ in range(4):
    next(it)

StopIteration: 

In [7]:
l = list(it)
print(l)

[]


Functions like `list` called on an iterator exhaust it, that is, go through all of its elements.

In [8]:
it = iter([2, 3, 4, 5, 6])
next(it)

2

In [9]:
list(it)

[3, 4, 5, 6]

In [10]:
next(it)

StopIteration: 

### Generators

In [11]:
def intSeq(n):
    i = 0
    while i < n:
        yield i
        i += 1

In [12]:
seq = intSeq(5)

`seq` is an iterator, `intSeq` is a generator function.

In [13]:
print(next(seq))

0


In [14]:
for i in seq:
    print(i)

1
2
3
4


In [15]:
next(seq)

StopIteration: 

In [16]:
seq = intSeq(5)
print(list(seq))
next(seq)

[0, 1, 2, 3, 4]


StopIteration: 

In [17]:
def intSeq2(n):
    i = 0
    while True:
        yield i
        i += 1
        if i >= n:
            raise StopIteration

In [18]:
seq2 = intSeq2(5)

In [19]:
for i in seq2:
    print(i)

0
1
2
3
4


RuntimeError: generator raised StopIteration

In [20]:
def intSeq3(n):
    i = 0
    while True:
        yield i
        i += 1
        if i >= n:
            return

In [21]:
seq3 = intSeq3(5)
for i in seq3:
    print(i)

0
1
2
3
4


In [22]:
def generator():
    try:
        yield 'fun'
        raise StopIteration
    except:
        yield 'glory'

In [23]:
gen = generator()
print(next(gen), '/', next(gen))

fun / glory


### A bug example

In [24]:
from functools import partial

In [25]:
square = lambda x: x**2

squares = partial(map, square)

In [26]:
avg = lambda seq: sum(seq) / len(seq)
res = lambda seq: avg(squares(seq))

`len` doesn't work with iterators. So we need to write our own length function that would work with iterators.

In [27]:
constOne = lambda _: 1
from toolz import compose
length = compose(sum, partial(map, constOne))

The problem is that the iterator will be exhausted.

In [28]:
from operator import truediv as div
from functools import reduce

In [29]:
avg = lambda seq: div(
    *reduce(lambda t, xs: (t[0] + xs, t[1] + 1), seq, (0, 0))
)
print(res([1, 2, 3]))

4.666666666666667


### Why would I use all this stuff?

In [30]:
def numbers():
    n = 0
    while True:
        yield n
        n += 1

In [31]:
N = numbers()

In [32]:
next(N)

0

In [33]:
next(N)

1

In [34]:
next(N)

2

In [35]:
next(N)

3

In [36]:
squares = map(square, numbers())

In [37]:
next(squares)

0

In [38]:
next(squares)

1

In [39]:
next(squares)

4

In [40]:
next(squares)

9

In [41]:
evens = (n for n in numbers() if not n % 2)

In [42]:
next(evens)

0

In [43]:
next(evens)

2

In [44]:
next(evens)

4

### Lazy itertools

In [45]:
from itertools import tee, count, cycle, repeat, islice

`tee` creates two independent copies of some iterator. Independent means that iterating over one of them doesn't change the other.

In [46]:
N = numbers()

In [47]:
N = numbers()
next(N)
next(N)
next(N)

2

This is why we use `tee`:

In [48]:
S = map(square, N)
next(S)

9

`count` is simply a generator equal to our `numbers` generator - that is, it returns an iterator containing the infinite sequence of integers starting from zero.

`cycle` takes an iterable and returns an iterator that repeats the elements of the iterable infinitely.

In [49]:
oneTwoThrees = cycle([1, 2, 3])
for _ in range(10):
    print(next(oneTwoThrees))

1
2
3
1
2
3
1
2
3
1


`repeat` takes one element and returns a generator that repeats this element infinitely.

In [50]:
AAAAA = repeat('A')
for _ in range(5):
    print(next(AAAAA))

A
A
A
A
A


`islice` is a lazy version of list slicing.

In [51]:
firstEvenSquares = lambda n: islice(squares, 0, n, 2)
print(list(firstEvenSquares(5)))

[16, 36, 64]


### Send and yield from
Instead of getting a value from the generator we may send a value to it.

In [52]:
def counter():
    cnt = 0
    while True:
        new = yield cnt
        print('new: ', new)
        if new:
            cnt = new
        else: 
            cnt += 1

We can use a special method `send` to set the value of the thing on the left of the `yield` keyword to some value. The `yield` statement doesn't only set the value of new to that something but it also yields this something as the current result of the generator function.

In [53]:
c = counter()
print(next(c))
print(next(c))
print(next(c))
print(next(c))
print(c.send(1))
print(next(c))

0
new:  None
1
new:  None
2
new:  None
3
new:  1
1
new:  None
2


In [54]:
def limCounter(n):
    cnt = 0
    while cnt < n:
        new = yield cnt
        if new:
            cnt = new
        else:
            cnt += 1

Let's make a new generator function that's running the first iterator, then once it's exhausted, it runs the second iterator. That is exactly what `yield from` enables us to do.

In [55]:
def doubleLimCounter():
    yield from limCounter(5)
    yield from limCounter(7)

In [56]:
d = doubleLimCounter()
print(next(d))
print(next(d))
print(next(d))
print(next(d))
print(next(d))
print(d.send(2))
print(next(d))
print(next(d))
print(next(d))
print(next(d))
print(next(d))
print(d.send(1))
print(next(d))

0
1
2
3
4
2
3
4
0
1
2
1
2


### Our own toolbox

Lazy functions on iterators.

In [57]:
take = lambda n, it: islice(it, 0, n)
drop = lambda n, it: islice(it, n, None)

In [58]:
head = partial(take, 1)
tail = partial(drop, 1)

In [59]:
force = compose(list, islice)

In [60]:
def iterate(f, x):
    yield x
    yield from (f, f(x))

But there's a recursive call and hence we're limited to about a thousand of elements. So, it's more preferable to avoid using recursion because iterate is frequently used to generate large sequences.

In [61]:
from itertools import accumulate

iterate = lambda f, x: accumulate(repeat(x), lambda a, b: f(a))

In [62]:
inc = lambda x: x + 1
numbers2 = iterate(inc, 0)
print(next(numbers2))
print(next(numbers2))

0
1


Church numerals:

In [63]:
zero = lambda f: lambda x: x
succ = lambda n: lambda f: lambda x: f(n(f)(x))
f = lambda x: x + 1
x = 0

In [64]:
numerals = iterate(succ, zero)
print(next(numerals)(f)(x))
print(next(numerals)(f)(x))
print(next(numerals)(f)(x))
print(next(numerals)(f)(x))
print(next(numerals)(f)(x))

0
1
2
3
4


### Some important examples

The sieve of Eratosthenes.

In [65]:
def eratoSieve(seq):
    currentPrime = next(seq)
    
    yield currentPrime
    
    sieved = filter(lambda x: x % currentPrime != 0, seq)
    
    yield from eratoSieve(sieved)

In [66]:
primes = eratoSieve(count(2))

In [67]:
print(next(primes))
print(next(primes))
print(next(primes))
print(next(primes))

2
3
5
7


Haskell version of Fibonacci numbers sequence generator.

In [68]:
zipWith = lambda tfunc: compose(partial(map, tfunc), zip)

def fibs():
    yield 1
    yield 1
    
    fibs1, fibs2 = tee(fibs())
    
    yield from zipWith(sum)(fibs1, tail(fibs2))

In [69]:
f = fibs()
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))

1
1
2
3
5
8
13
21
34
55


Series expansion for $e^{x}$:

In [70]:
def inc_improve(seq, eps):
    
    def convergeTest(prev, curr):
        if abs(prev - curr) < eps:
            raise StopIteration
        else:
            return curr
        
    return accumulate(seq, convergeTest)

def expSeries(x):
    i = 1
    res = 1
    term = 1
    
    while True:
        yield res
        
        term *= (x / i)
        i += 1
        res += term

In [71]:
eSeq = inc_improve(expSeries(1), 1e-15)
for app in eSeq:
    print(app)

1
2.0
2.5
2.6666666666666665
2.708333333333333
2.7166666666666663
2.7180555555555554
2.7182539682539684
2.71827876984127
2.7182815255731922
2.7182818011463845
2.718281826198493
2.7182818282861687
2.7182818284467594
2.71828182845823
2.718281828458995
2.718281828459043
2.7182818284590455
